In [ ]:
from music21 import *
import collections
import numpy as np
import pandas as pd
import gzip
import random

In [ ]:
us = environment.UserSettings()
us['musicxmlPath'] = 'C:/Program Files/MuseScore 3/bin/MuseScore3.exe'
us['musescoreDirectPNGPath'] = 'C:/Program Files/MuseScore 3/bin/MuseScore3.exe'

### Get corpus and parse to pieces

In [ ]:
coreCorpus = corpus.corpora.CoreCorpus()
sixEight = corpus.search('6/8')

bachCorpusScores = []
for c in sixEight:
    score = c.parse()
    bachCorpusScores.append(score)

### Collect notes from a single piece

In [ ]:
sBach = corpus.parse('bach/bwv57.8')

In [ ]:
len(sBach.getElementsByClass(stream.Part))

measures = len(sBach.getElementsByClass(stream.Part)[0].getElementsByClass(stream.Measure))
# Get a part of the piece
noteIterator = sBach.parts[0].getElementsByClass(stream.Measure).flat.getElementsByClass('Note')
allNotes = []

for el in noteIterator:
    if('-' in el.nameWithOctave):
        noteName = el.pitch.getEnharmonic().nameWithOctave
        allNotes.append(noteName)
    else:
        noteName = el.nameWithOctave
        allNotes.append(noteName)

# Collect notes from a corpus

In [ ]:
possibleNotes = set() # set containing all possible notes for matrix creation
allNotesPerPiece = [] # Multidimensional array of all notes per piece
for p in bachCorpusScores:
    currNotes = []
    measures = len(p.getElementsByClass(stream.Part)[0].getElementsByClass(stream.Measure))
    # Get a part of the piece
    noteIterator = p.parts[0].getElementsByClass(stream.Measure).flat.getElementsByClass('Note')
    if(len(noteIterator) == 0):
        continue
    for el in noteIterator:
        pitchName = el.nameWithOctave
        if('-' in pitchName or '##' in pitchName):
            noteName = el.pitch.getEnharmonic().nameWithOctave
            possibleNotes.add(noteName)
            currNotes.append(noteName)
        else:
            noteName = el.nameWithOctave
            possibleNotes.add(noteName)
            currNotes.append(noteName)
            
    allNotesPerPiece.append(currNotes)

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]
flatten(allNotesPerPiece)

In [ ]:
# Get frequency array
counter = collections.Counter(flatten(allNotesPerPiece))


In [ ]:
# Check if no empty pieces
print(len(allNotesPerPiece))
print(len(list(filter(lambda x: x > 0, map(len, allNotesPerPiece)))))


### Create frequency matrix

In [ ]:
# Get frequency array
counter = collections.Counter(flatten(allNotesPerPiece))
# Initial note counter
counter[' '] = len(allNotesPerPiece)

noteRows = possibleNotes.add(' ')

zeros = np.full((len(possibleNotes), len(possibleNotes)), 0)
matrix = pd.DataFrame(zeros, index=possibleNotes, columns=possibleNotes)
matrix = matrix.astype(float)

for allNotes in allNotesPerPiece:
    # Fill transition matrix frequencies
    for i in range(len(allNotes)+1):
        # First note
        if(i == 0):
            matrix[' '][allNotes[i]] = matrix[' '][allNotes[i]] + 1
            continue
        # Last note
        if(i == len(allNotes)):
            matrix[allNotes[i-1]][' '] = matrix[allNotes[i-1]][' '] + 1
            continue

        currNote = allNotes[i-1]
        nextNote = allNotes[i]

        matrix[currNote][nextNote] = matrix[currNote][nextNote] + 1


In [ ]:
### Divide each row to get probabilistic model
for i in possibleNotes:
    for j in possibleNotes:
        matrix[j][i] = matrix[j][i] / counter[j]

In [ ]:
tr = matrix[' ']
tr

In [ ]:
# Check if p sum up to one
matrix.sum(0)

In [ ]:
def getRandomTransition(matrix, startNote):
    rng = random.random()
    transitions = []
    if(startNote is None):
        transitions = matrix[' ']
    else:
        transitions = matrix[startNote]
    pSum = 0.0
    for k,v in transitions.iteritems():
        if rng > pSum and rng < pSum + v:
            return k
        else:
            pSum += v
        
    print(f'P of {startNote} does not add up to 1.0')
    print(f'Sum = {pSum}')
    print(f'Rng = {rng}')        

In [ ]:
populationSize = 10
iterations = 1
measureLength = 8

population = []

for i in range(populationSize): 
    individual = []
    for j in range(measureLength):
        random.random()
        while(len(individual) < measureLength):
            nextNote = None
            if(len(individual) == 0):
                # Add durations
                nextNote = getRandomTransition(matrix, None)
            else:
                # Add durations
                nextNote = getRandomTransition(matrix, individual[-1])
            individual.append(nextNote)
    print(individual)
    population.append(individual)




In [ ]:
# Map note strings to music21 notes
notes = list(map(lambda x: list(map(lambda y: note.Note(y), x)), population))
s = stream.Score(id='mainScore')
part = stream.Part(id='part0')
measures = []
for i in range(notes):
    m = stream.Measure(i+1)
    notesInMeasure = notes[i]
    m.extend(notesInMeasure)
    part.append(m)
    
part

In [ ]:
# Export to csv
matrix.to_csv(path_or_buf="./transition-matrix.csv")